In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
# Specify sub_step parameters
substep_params={
}

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME, get_tmp_work_path

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
    # tmp_entities =
    # [
    #     { ENTITY_NAME: "tmp_dir_to_store" },
    #     { ENTITY_NAME: "tmp_dir_to_load" }
    # ],
    
    # custom_inputs = 
    # [
    #     { ENTITY_NAME: "big_file", ENTITY_PATH: "/data/tmp/user/pipeline/zone/big_file" }
    # ],
    
    outputs = 
    [
        { ENTITY_NAME: "stored_files" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
import zlib

def crc32(fileName):
    with open(fileName, 'rb') as fh:
        hash = 0
        while True:
            s = fh.read(65536)
            if not s:
                break
            hash = zlib.crc32(s, hash)
        return "%08X" % (hash & 0xFFFFFFFF)

In [ ]:
# create test data
import os, random, string
from pathlib import Path
#tmp_entities = substep.tmp_entities()

tmp_dir_to_store = get_tmp_work_path()

for i in range(1, 10):
    with open(f'{tmp_dir_to_store}/file{i}.txt', 'w') as f:
        f.write(''.join(random.choices(string.ascii_lowercase, k=i*10))) # generate random content file size of i*10

with open(f'{tmp_dir_to_store}/big_file.bin', 'wb') as f:
    f.write(os.urandom(1000000))    # generate random content file larger than ROW_SIZE
crc1 = crc32(f'{tmp_dir_to_store}/big_file.bin')

Path(tmp_dir_to_store, "subdir").mkdir(parents=True, exist_ok=True)

with open(f'{tmp_dir_to_store}/subdir/sub_big_file.bin', 'wb') as f:
    f.write(os.urandom(1000000))    # generate random content file larger than ROW_SIZE in subdir
crc1 = crc32(f'{tmp_dir_to_store}/subdir/sub_big_file.bin')

In [ ]:
from sinara.archive import SinaraArchive
arhive = SinaraArchive(spark)

In [ ]:
import glob
import os

outputs = substep.outputs()

stored_files = [os.path.basename(x) for x in glob.glob(tmp_dir_to_store + '/**', recursive=True)]
print(stored_files)

arhive.pack(tmp_dir_to_store, outputs.stored_files)

In [ ]:
tmp_dir_to_load = arhive.unpack(outputs.stored_files)
loaded_files = [os.path.basename(x) for x in glob.glob(tmp_dir_to_load + '/**', recursive=True)]
print(loaded_files)

In [ ]:
assert set(stored_files) == set(loaded_files), "stored and loaded files are not equal"

crc2 = crc32(f'{tmp_dir_to_load}/big_file.bin')

assert set(crc1) == set(crc1), "stored and loaded big files files are not equal"

In [ ]:
SinaraSpark.stop_session()